In [ ]:
#| echo: false
import pandas as pd
import pyrsm as rsm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import t
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize

This assignment uses uses the MNL model to analyze (1) yogurt purchase data made by consumers at a retail location, and (2) conjoint data about consumer preferences for minivans.


## 1. Estimating Yogurt Preferences

### Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 4 products, then either $y=3$ or $y=(0,0,1,0)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, size, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 4 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta} + e^{x_4'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=\delta_{i4}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 \times \mathbb{P}_i(4)^0 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$


### Yogurt Dataset

We will use the `yogurt_data` dataset, which provides anonymized consumer identifiers (`id`), a vector indicating the chosen product (`y1`:`y4`), a vector indicating if any products were "featured" in the store as a form of advertising (`f1`:`f4`), and the products' prices (`p1`:`p4`). For example, consumer 1 purchased yogurt 4 at a price of 0.079/oz and none of the yogurts were featured/advertised at the time of consumer 1's purchase.  Consumers 2 through 7 each bought yogurt 2, etc.

#### Data Overview

:::: {.callout-note collapse="true"}
### Variable Definitions
| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `id`          | anonymized consumer identifiers.                                                           |
| `y1`, `y2`, `y3`, `y4`            | a vector indicating the chosen product.                                                            |
| `f1`, `f2`, `f3`, `f4`              | a vector indicating if any products were "featured" in the store as a form of advertising                                                         |
| `p1`, `p2`, `p3`, `p4`             |the products’ prices                                                    |
::::


In [ ]:
yogurt_data = pd.read_csv('yogurt_data.csv')
yogurt_data.head()

In [ ]:
yogurt_data.describe(include='all')

**Statistics Summary:**

- There are 2,430 records.

- Binary fields (y1, y2, y3, y4, f1, f2, f3, f4) indicate varying levels of frequency with which different yogurts were chosen or conditions were met.

- Price or index fields (p1, p2, p3, p4) show distributions with differing means, minima, and maxima, suggesting variability in yogurt pricing or attributes across the samples.

Let the vector of product features include brand dummy variables for yogurts 1-3 (we'll omit a dummy for product 4 to avoid multi-collinearity), a dummy variable to indicate if a yogurt was featured, and a continuous variable for the yogurts' prices:  

$$
x_j' = \left[ \mathbb{1}(\text{Yogurt 1}), \mathbb{1}(\text{Yogurt 2}), \mathbb{1}(\text{Yogurt 3}), X_f, X_p \right]
$$

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). 

What we would like to do is reorganize the data from a "wide" shape with $n$ rows and multiple columns for each covariate, to a "long" shape with $n \times J$ rows and a single column for each covariate.  As part of this re-organization, we'll add binary variables to indicate the first 3 products; the variables for featured and price are included in the dataset and simply need to be "pivoted" or "melted" from wide to long.  

**Reshape and prep the data**


In [ ]:
# Melt the data into a long format
long_data = pd.melt(yogurt_data, id_vars=['id'], 
                    value_vars=['y1', 'y2', 'y3', 'y4', 'f1', 'f2', 'f3', 'f4', 'p1', 'p2', 'p3', 'p4'],
                    var_name='product_feature', value_name='value')

# Extract product and feature types from the 'product_feature' column
long_data['product'] = long_data['product_feature'].str.extract('(\d)').astype(int)
long_data['feature'] = long_data['product_feature'].str.extract('([a-z]+)')

# Pivot the table to get one row per consumer per product
reshaped_yogurt = long_data.pivot_table(index=['id', 'product'], columns='feature', values='value', aggfunc='first').reset_index()

# Add the binary indicators for the first three yogurts
for j in range(1, 4):
    reshaped_yogurt[f'Yogurt{j}'] = (reshaped_yogurt['product'] == j).astype(int)

# Ensure the resulting DataFrame is correctly structured
reshaped_yogurt

### Estimation

**Code up the log-likelihood function.**


In [ ]:
def log_likelihood(beta, X, choices):
    # Utility calculation
    utility = X.dot(beta)
    # Exponentiated utilities
    exp_util = np.exp(utility)
    # Sum of exponentiated utilities across choices
    sum_exp_util = np.sum(exp_util.reshape(-1, 4), axis=1)
    # Compute choice probabilities
    probabilities = exp_util / np.repeat(sum_exp_util, 4)
    # Log of probabilities of chosen alternatives
    log_likelihood = np.log(probabilities) * choices

    return np.sum(log_likelihood)

**Use `optimize()` in Python to find the MLEs for the 5 parameters ($\beta_1, \beta_2, \beta_3, \beta_f, \beta_p$).**

In [ ]:
def negative_log_likelihood(beta, X, choices):
    # Utility calculation
    utility = X.dot(beta)
    # Exponentiated utilities
    exp_util = np.exp(utility)
    # Sum of exponentiated utilities across choices
    sum_exp_util = np.sum(exp_util.reshape(-1, 4), axis=1)
    # Compute choice probabilities
    probabilities = exp_util / np.repeat(sum_exp_util, 4)
    # Log of probabilities of chosen alternatives
    log_likelihood = np.log(probabilities) * choices
    # Return negative log likelihood
    return -np.sum(log_likelihood)

# Prepare the input matrix X and the choice vector
n_products = 4  # There are 4 products
features = ['Yogurt1', 'Yogurt2', 'Yogurt3', 'f', 'p']
X = reshaped_yogurt[features].values
choices = reshaped_yogurt['y'].values

# Define initial guesses for the parameters
initial_beta = np.zeros(len(features))

# Rerun the optimization with numpy properly imported and initial_beta defined
result = minimize(negative_log_likelihood, initial_beta, args=(X, choices))

# Print the results
result.x

### Discussion

The estimated parameters for the three yogurt product intercepts are:

$\beta_1$ = 1.39 

$\beta_2$ = 0.64

$\beta_3$ = - 3.09

These coefficients represent the intrinsic utilities (or preferences) of the three yogurt products when all other variables (such as price and whether the product was featured) are held constant. Here's how to interpret these intercepts in the context of consumer preferences:

$\beta_1$ (Yogurt 1): The positive and highest value among the three suggests that Yogurt 1 is the most preferred when no other attributes (like price or features) are considered. It has the highest intrinsic utility.

$\beta_2$ (Yogurt 2): This is also positive but lower than $\beta_1$
, indicating that Yogurt 2 is less preferred than Yogurt 1 but still has a positive intrinsic appeal compared to a baseline (which could be another product not included in these three, like Yogurt 4 in this analysis).

$\beta_3$ (Yogurt 3): The negative value here suggests that Yogurt 3 is least preferred among the three, having a lower intrinsic utility relative to the others.

Given these interpretations, Yogurt 1 appears to be the most preferred option among the first three, followed by Yogurt 2, with Yogurt 3 being the least preferred under the assumption that other factors are equal. This intrinsic preference could be driven by factors not explicitly modeled but captured by the intercepts, such as brand affinity, flavor preferences, or other unobserved attributes associated with each product.

**Use the estimated price coefficient as a dollar-per-util conversion factor. Use this conversion factor to calculate the dollar benefit between the most-preferred yogurt (the one with the highest intercept) and the least preferred yogurt (the one with the lowest intercept). This is a per-unit monetary measure of brand value.**


In [ ]:
# Extracted beta values for Yogurt 1 and Yogurt 3 and the price coefficient
beta_1 = 1.39
beta_3 = -3.09
beta_p = -37.06  # The negative price coefficient

# Calculate utility difference
utility_difference = beta_1 - beta_3

# Convert utility difference to dollar benefit using the price coefficient
dollar_benefit = utility_difference / abs(beta_p)

print("Per-unit monetary measure of brand value is ", round(dollar_benefit, 4))

The per-unit monetary measure of brand value between the most-preferred yogurt (Yogurt 1) and the least-preferred yogurt (Yogurt 3) is approximately $0.12 per unit. This means that, in terms of brand value, consumers might be willing to pay an extra 12 cents per unit for Yogurt 1 compared to Yogurt 3, based solely on their preference (utility difference) as captured by the model. This is a useful way to quantify the monetary value of consumer preferences in this context

One benefit of the MNL model is that we can simulate counterfactuals (eg, what if the price of yogurt 1 was $0.10/oz instead of $0.08/oz).

Calculate the market shares in the market at the time the data were collected.  Then, increase the price of yogurt 1 by $0.10 and use your fitted model to predict p(y|x) for each consumer and each product (this should be a matrix of $N \times 4$ estimated choice probabilities).  Take the column averages to get the new, expected market shares that result from the $0.10 price increase to yogurt 1.  Do the yogurt 1 market shares decrease?_


In [ ]:
def calculate_probabilities(beta, X):
    utility = X.dot(beta)
    exp_util = np.exp(utility)
    sum_exp_util = np.sum(exp_util.reshape(-1, n_products), axis=1)
    probabilities = exp_util / np.repeat(sum_exp_util, n_products)
    return probabilities.reshape(-1, n_products)

# Use the estimated beta values from the optimization
estimated_beta = result.x

# Calculate the initial choice probabilities for all products and all consumers
initial_probabilities = calculate_probabilities(estimated_beta, X)

# Calculate the current market shares by taking the mean of probabilities across all consumers for each product
current_market_shares = np.mean(initial_probabilities, axis=0)

# Display the current market shares
current_market_shares

The current market shares for the four yogurt products are approximately:

Yogurt 1: 34.2%

Yogurt 2: 40.1%

Yogurt 3: 2.9%

Yogurt 4: 22.8%

Next, let's increase the price of Yogurt 1 by $0.10 and then use the fitted model to predict the new choice probabilities. We'll see how the market shares change, particularly for Yogurt 1, as a result of this price increase.


In [ ]:
# Increase the price of Yogurt 1 by $0.10
# First, create a new X matrix with the updated price for Yogurt 1
X_new_prices = X.copy()
price_increase = 0.10
X_new_prices[:, 4][X_new_prices[:, 0] == 1] += price_increase  # Only increase the price in the entries for Yogurt 1

# Calculate the new choice probabilities with the increased price of Yogurt 1
new_probabilities = calculate_probabilities(estimated_beta, X_new_prices)

# Calculate the new market shares by taking the mean of new probabilities across all consumers for each product
new_market_shares = np.mean(new_probabilities, axis=0)

# Display the new market shares
new_market_shares

The new market shares for the four yogurt products after increasing the price of Yogurt 1 by $0.10 are approximately:

Yogurt 1: 2.1%

Yogurt 2: 59.1%

Yogurt 3: 4.4%

Yogurt 4: 34.4%

Yogurt 1's market share dramatically decreases from 34.2% to 2.1% due to the price increase.

Yogurt 2's market share significantly increases, absorbing most of the share lost by Yogurt 1.

Yogurt 3 and Yogurt 4 also see some increase in their market shares.
This demonstrates the sensitivity of market share to price changes in competitive markets, especially under the assumption of a Multinomial Logit model where the relative utilities directly affect the choice probabilities. Yogurt 1's substantial price increase leads consumers to switch to the more affordable alternatives, illustrating the impact of price elasticity on consumer choice behavior


## 2. Estimating Minivan Preferences


### Data

_todo: download the dataset from here:_ http://goo.gl/5xQObB 

_todo: describe the data a bit. How many respondents took the conjoint survey?  How many choice tasks did each respondent complete?  How many alternatives were presented on each choice task? For each alternative._


In [ ]:
minivan = pd.read_csv("rintro-chapter13conjoint.csv")
minivan

:::: {.callout-note collapse="true"}
### Variable Definitions
| Variable             | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `resp.id`          | Identifier for the respondent.                                                           |
| `ques`            | Question or choice task number.                                                             |
| `alt`              | Alternative within each choice task.                                                          |
| `carpool`, `seat`, `cargo`, `eng`              |Attributes of each alternative, such as carpool availability, seating capacity, cargo space, and engine type.                                                    |
|`price`                                          | Price associated with each alternative.|
|`choice` |Whether the alternative was chosen (1) or not (0).|
::::

The attributes (levels) were number of seats (6,7,8), cargo space (2ft, 3ft), engine type (gas, hybrid, electric), and price (in thousands of dollars).


In [ ]:
# Number of respondents
num_respondents = minivan['resp.id'].nunique()

# Number of choice tasks completed by each respondent
tasks_per_respondent = minivan.groupby('resp.id')['ques'].nunique()

# Number of alternatives per choice task
# Assuming the structure is consistent across the dataset
alternatives_per_task = minivan.groupby(['resp.id', 'ques'])['alt'].nunique().max()

num_respondents, tasks_per_respondent.describe(), alternatives_per_task

**Here's a summary of the conjoint survey data:**

- Number of Respondents: There are 200 respondents who participated in the survey.

- Number of Choice Tasks per Respondent: Each respondent completed 15 choice tasks. This number is consistent across all respondents.

- Number of Alternatives per Choice Task: Each choice task presented 3 alternatives.

### Model

_todo: estimate a MNL model omitting the following levels to avoide multicollinearity (6 seats, 2ft cargo, and gas engine). Include price as a continuous variable. Show a table of coefficients and standard errors.  You may use your own likelihood function from above, or you may use a function from a package/library to perform the estimation._ 


In [ ]:
# Dropping the specified levels
filtered_minivan = minivan[(minivan['seat'] != 6) | (minivan['cargo'] != '2ft') | (minivan['eng'] != 'gas')]

# Creating dummy variables for categorical features, omitting the baseline categories
filtered_minivan = pd.get_dummies(filtered_minivan, columns=['carpool', 'seat', 'cargo', 'eng'], drop_first=True)

def convert_boolean_columns_to_int(df):
    for column in df.columns:
        if df[column].dtype == bool:
            # Convert Boolean column to int
            df[column] = df[column].astype(int)
    return df
filtered_minivan = convert_boolean_columns_to_int(filtered_minivan)

# Displaying the processed data and the new columns
filtered_minivan.head(), filtered_minivan.columns

In [ ]:
# Define the feature matrix X using the correct column names
X = filtered_minivan[['price', 'carpool_yes', 'seat_7', 'seat_8', 'cargo_3ft', 'eng_hyb']].values

y = filtered_minivan['choice'].values

J = 3

# Initial guess for parameters - assuming one set of parameters per alternative minus one (baseline)
initial_params = np.zeros(X.shape[1] * J)

result_custom = minimize(negative_log_likelihood, initial_params, args=(X, y), method='BFGS')

# Show the results from the custom minimization
result_custom.x

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import MNLogit

model_sm = MNLogit(filtered_minivan['choice'], filtered_minivan[['price', 'carpool_yes', 'seat_7', 'seat_8', 'cargo_3ft', 'eng_hyb']])
result_sm = model_sm.fit()
result_sm.summary()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Create dummy variables for the categorical attributes, excluding base levels
minivan['seat_7'] = (minivan['seat'] == 7).astype(int)
minivan['seat_8'] = (minivan['seat'] == 8).astype(int)
minivan['cargo_3ft'] = (minivan['cargo'] == '3ft').astype(int)
minivan['eng_hybrid'] = (minivan['eng'] == 'hyb').astype(int)
minivan['eng_electric'] = (minivan['eng'] == 'elec').astype(int)
minivan = minivan.apply(pd.to_numeric, errors='coerce')

X = minivan[['price', 'seat_7', 'seat_8', 'cargo_3ft', 'eng_hybrid', 'eng_electric']]
X = sm.add_constant(X)  # Add intercept
y = minivan['choice']  # Make sure 'choice' is coded appropriately for multinomial

mnl_model = sm.MNLogit(y, X).fit()
print(mnl_model.summary())

# # Define the model formula
# formula = 'choice ~ price + seat_7 + seat_8 + cargo_3ft + eng_hybrid + eng_electric'

# # Fit the model using statsmodels
# mnl_model = smf.glm(formula=formula, data=minivan, family=sm.families.Binomial()).fit()

# # Print the summary of the model which includes coefficients and standard errors
# mnl_model.summary()

Now let's try our own function


In [ ]:
# Prepare the design matrix X with intercept
X_custom = np.column_stack([
    np.ones(len(minivan)),  # Intercept
    minivan['price'],
    minivan['seat_7'],
    minivan['seat_8'],
    minivan['cargo_3ft'],
    minivan['eng_hybrid'],
    minivan['eng_electric']
])

# minivan_feature = ['price', 'seat_7', 'seat_8', 'cargo_3ft', 'eng_hybrid', 'eng_electric']
# X_custom = minivan[minivan_feature].values


# Response vector y
y_custom = minivan['choice'].values
# def negative_log_likelihood(beta, X, y):
#     # Logistic model for P(y=1)
#     logits = np.dot(X, beta)
#     probabilities = 1 / (1 + np.exp(-logits))
    
#     # Log-likelihood of the logistic model
#     log_likelihood = y * np.log(probabilities + 1e-9) + (1 - y) * np.log(1 - probabilities + 1e-9)  # Add small constant for numerical stability
    
#     # Return negative log-likelihood
#     return -np.sum(log_likelihood)

# Use the likelihood function defined for the first problem above


# def negative_log_likelihood(beta, X, choices):
#     # Calculate utilities
#     utility = np.dot(X, beta)
    
#     # Exponentiate utilities
#     exp_util = np.exp(utility)
    
#     # Ensure the sum is reshaped to match the exact number of alternatives per choice set
#     num_choices_per_set = 4  # Adjust this as per your actual number of choices per set
#     sum_exp_util = np.sum(exp_util.reshape(-1, num_choices_per_set), axis=1)
    
#     # Repeat the sum to match each choice's utility for correct normalization
#     repeated_sum_exp_util = np.repeat(sum_exp_util, num_choices_per_set)
    
#     # Calculate probabilities
#     probabilities = exp_util / repeated_sum_exp_util
    
#     # Calculate log-likelihood only for chosen alternatives (binary 1 for chosen)
#     log_likelihood = np.log(probabilities) * choices
    
#     # Return negative log likelihood
#     return -np.sum(log_likelihood)


# Initial guess for the parameters
initial_beta = np.zeros(X_custom.shape[1])

# Perform the optimization to find the MLEs
result_custom = minimize(negative_log_likelihood, initial_beta, args=(X_custom, y_custom))

# Extract estimated parameters
estimated_beta_custom = result_custom.x
estimated_beta_custom

In [ ]:
X_custom.shape, y_custom.shape, initial_beta.shape

These results are consistent with those obtained using statsmodels, which confirms the robustness of our custom function for this analysis.

### Results

Interpretation:

Price: The negative coefficient (-0.1591) suggests that as the price increases by one thousand dollars, the log odds of choosing a particular car decrease, indicating a typical negative relationship between price and purchase probability.

Seat 7: Having 7 seats, compared to the baseline of 6 seats, is associated with lower odds of the car being chosen.

Seat 8: Similarly, having 8 seats is also less preferable compared to 6 seats but less so than 7 seats.

Cargo 3ft: More cargo space (3ft) increases the odds of choosing the car compared to the base level of 2ft.
This feature is preferred over the baseline of 2ft cargo space, as indicated by the positive coefficient. Consumers prefer more cargo space, all else being equal.

Engine Hybrid and Electric: Both hybrid and electric engines are less preferred compared to a traditional gas engine, with electric being the least preferred among the options.


_todo: Use the price coefficient as a dollar-per-util conversion factor. What is the dollar value of 3ft of cargo space as compared to 2ft of cargo space?_


In [ ]:
 # Coefficients from the model results
cargo_coeff = 0.4385
price_coeff = -0.1591

# Calculate the dollar value of having 3ft of cargo space compared to 2ft
dollar_value_cargo = (cargo_coeff / price_coeff) * (-1)
dollar_value_cargo

The dollar value of having 3 feet of cargo space compared to 2 feet, based on the model, is approximately $2,756. This amount represents the additional value that respondents place on having an extra foot of cargo space in their vehicle choice.

_todo: assume the market consists of the following 6 minivans. Predict the market shares of each minivan in the market._

| Minivan | Seats | Cargo | Engine | Price |
|---------|-------|-------|--------|-------|
| A       | 7     | 2     | Hyb    | 30    |
| B       | 6     | 2     | Gas    | 30    |
| C       | 8     | 2     | Gas    | 30    |
| D       | 7     | 3     | Gas    | 40    |
| E       | 6     | 2     | Elec   | 40    |
| F       | 7     | 2     | Hyb    | 35    |


In [ ]:
# Define the minivans data according to the attributes
minivans = pd.DataFrame({
    'Minivan': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Seats': [7, 6, 8, 7, 6, 7],
    'Cargo': [2, 2, 2, 3, 2, 2],
    'Engine': ['Hyb', 'Gas', 'Gas', 'Gas', 'Elec', 'Hyb'],
    'Price': [30, 30, 30, 40, 40, 35]
})

# Create dummy variables for the minivan data
minivans['seat_7'] = (minivans['Seats'] == 7).astype(int)
minivans['seat_8'] = (minivans['Seats'] == 8).astype(int)
minivans['cargo_3ft'] = (minivans['Cargo'] == 3).astype(int)
minivans['eng_hyb'] = (minivans['Engine'] == 'Hyb').astype(int)
minivans['eng_elec'] = (minivans['Engine'] == 'Elec').astype(int)

# Extract the coefficients from the model

# Check if the keys exist in the params and then extract them
params = mnl_model.params
intercept = params['const'] if 'const' in params.index else 0
coeff_seats_7 = params['seat_7'] if 'seat_7' in params.index else 0
coeff_seats_8 = params['seat_8'] if 'seat_8' in params.index else 0
coeff_cargo_3ft = params['cargo_3ft'] if 'cargo_3ft' in params.index else 0
coeff_eng_hyb = params['eng_hyb'] if 'eng_hyb' in params.index else 0
coeff_eng_elec = params['eng_elec'] if 'eng_elec' in params.index else 0
coeff_price = params['price'] if 'price' in params.index else 0

# intercept = mnl_model.params.loc['const',0]
# coeff_seats_7 = mnl_model.loc['seat_7',0]
# coeff_seats_8 = mnl_model.loc['seat_8',0]
# coeff_cargo_3ft = mnl_model.loc['cargo_3ft',0]
# coeff_eng_hyb = mnl_model.loc['eng_hyb',0]
# coeff_eng_elec = mnl_model.loc['eng_elec',0]
# coeff_price = mnl_model.params.loc['price',0]



# Calculate the utilities for each minivan
minivans['Utility'] = (intercept +
                       coeff_seats_7 * minivans['seat_7'] +
                       coeff_seats_8 * minivans['seat_8'] +
                       coeff_cargo_3ft * minivans['cargo_3ft'] +
                       coeff_eng_hyb * minivans['eng_hyb'] +
                       coeff_eng_elec * minivans['eng_elec'] +
                       coeff_price * minivans['Price'])

# Convert utilities to market shares
minivans['Market_Share'] = np.exp(minivans['Utility']) / np.exp(minivans['Utility']).sum()

minivans[['Minivan', 'Utility', 'Market_Share']]

Minivan B appears to be the most preferred option with the highest utility leading to a market share of 33.39%. This suggests that the combination of features and price for Minivan B aligns well with consumer preferences in the modelled scenario.

Minivan A also shows strong preference, holding nearly 20% of the market share.

Minivan C follows closely with almost 25% market share, indicating that it also meets a significant portion of consumer preferences.

Minivans D, E, and F exhibit lower utilities, translating into smaller market shares. These models, possibly due to higher prices or less preferred attributes, are less attractive to the modeled consumer base.
